<div style="width:100%; background-color: #000041"><a target="_blank\" href="http://university.yugabyte.com\"><img src="assets/YBU_Logo.webp" /></a></div><br>

> **YugabyteDB YCQL Development**
>
> Enroll for free at [Yugabyte University](https://university.yugabyte.com/courses/yugabytedb-ycql-development).
>

# Secondary indexes
In this notebook, you will learn how to create secondary indexes to not only improve query performance, but also remove unnecessary tables from the data model.

### Import the notebook variables 

> Requirements:
>
> You must first create the variables in the `01_Setup.ipynb` notebook.
>

The following Python cell reads the stored variables created in the `01_Setup.ipynb` notebook. 

- To run the script, select Execute Cell (Play Arrow) in the left gutter of the cell.

In [2]:
%store -r MY_DB_NAME
%store -r MY_YB_PATH
%store -r MY_YB_PATH_DATA
%store -r MY_GITPOD_WORKSPACE_URL
%store -r MY_HOST_IPv4_01
%store -r MY_HOST_IPv4_02
%store -r MY_HOST_IPv4_03
%store -r MY_NOTEBOOK_DIR
%store -r MY_TSERVER_WEBSERVER_PORT
%store -r MY_NOTEBOOK_DATA_FOLDER
%store -r MY_YB_MASTER_HOST_GITPOD_URL
%store -r MY_YB_TSERVER_HOST_GITPOD_URL
%store -r MY_DATA_DDL_FILE
%store -r MY_DATA_DML_FILE

---
## Secondary index: `Index Scan` query plan
In this section, you will create secondary indexes. A secondary index is a data structure that contains some of the columns of the index table and an index key that supports one or more data access patterns. 

To begin, examine the query plan without the implementation of a secondary index.

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME"   # new query plan
YB_PATH=${1}
DB_NAME=${2}  
cd $YB_PATH/bin

# DB_NAME=ks_ybu
./ycqlsh -r -k $DB_NAME -e "
  explain select  category, brand, product_name, description, product_id 
  from tbl_products_by_category 
  where brand=?;
"  

The query plan reveals that the query uses a sequence scan.

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME"   # Secondary index
YB_PATH=${1}
DB_NAME=${2}  
cd $YB_PATH/bin

# DB_NAME=ks_ybu
./ycqlsh -r -k $DB_NAME -e "
  describe tbl_products_by_category;
  "

### Create the secondary index

> **Important**
>
> You can only create a secondary index for a table with the `transactions` property enabled.
> 

To begin, drop the secondary index if it exists.

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME"   # Secondary index
YB_PATH=${1}
DB_NAME=${2}  
cd $YB_PATH/bin

# DB_NAME=ks_ybu
./ycqlsh -r -k $DB_NAME -e "
drop index if exists idx_products_by_category_brand;"

sleep 1;

Run the following cell to create an index by describing the `tbl_products_by_category` table.

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME"   # Secondary index
YB_PATH=${1}
DB_NAME=${2}  
cd $YB_PATH/bin

# DB_NAME=ks_ybu
./ycqlsh -r -k $DB_NAME -e "
create index if not exists idx_products_by_category_brand 
  on tbl_products_by_category (brand) 
  ;"

Use the `describe` keyword  to verify if the index was created for the given table. 


In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME"   # describe table to view index
YB_PATH=${1}
DB_NAME=${2}  
cd $YB_PATH/bin

# DB_NAME=ks_ybu
./ycqlsh -r -k $DB_NAME -e "
  describe tbl_products_by_category;
  "

The index key consists of a partition key and zero or more clustering keys. In the previous example, you can see that the index key is `brand`. The index inherits the primary key from the index table. The clustering keys are `category`,`product_name`, and `product_id`. 

### View query plan for a secondary index

> **Important:** 
> The primary key of the index table is part of the secondary index data structure. 

Run the following cell to verify if adding a secondary index for the `brand` can make the product_name query on the `tbl_products_by_category` more efficient.

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME"   # new query plan
YB_PATH=${1}
DB_NAME=${2}  
cd $YB_PATH/bin

# DB_NAME=ks_ybu
./ycqlsh -r -k $DB_NAME -e "
  explain select  category, brand, product_name, description, product_id 
  from tbl_products_by_category 
  where brand=?;
"  

#### Index Scan
With the introduction of a secondary index for the table, there is a new query plan. This query plan contains one node. The action is an index scan. Key Conditions is a sub-action that specifies the use of the partition key for the index. The index contains the partition key for the index and the primary key columns of the index table. After accessing the index, the query accesses the table. The table contains the description column. The action to access the index table is similar to a primary key lookup. Since the index contains the primary key columns, the query is able to lookup the rows in the table using the primary key values.

To learn more about why this is the case, take a look at the SST files for an index tablet.

### Index tablet and index scan query
YugabyteDB stores a secondary index in the same way as it does for a table. A secondary index exists as tablets. The data structure is also similar. It is DocDB. 

#### Select a YB-TServer host
Set the host variable for one of the nodes. All three nodes in the cluster are running a Tablet Server (YB-TServer). You can comment/uncomment lines 7-9 as needed.


In [ ]:
%%bash -s "$MY_HOST_IPv4_01" "$MY_HOST_IPv4_02" "$MY_HOST_IPv4_03" --out MY_HOST_IPv4
HOST_IPv4_01=$( echo "${1}" | tr -d " ")
HOST_IPv4_02=$( echo "${2}" | tr -d " ")
HOST_IPv4_03=$( echo "${3}" | tr -d " ")

# change the hosts for different tablet leaders
MY_HOST_IPv4=$HOST_IPv4_01
#MY_HOST_IPv4=$HOST_IPv4_02
#MY_HOST_IPv4=$HOST_IPv4_03

echo ${MY_HOST_IPv4}

Store the select host variable.

In [ ]:
%store MY_HOST_IPv4
print(MY_HOST_IPv4)

Save the table name as a variable.

In [ ]:
MY_OBJECT_NAME="idx_products_by_category_brand"
%store MY_OBJECT_NAME
print(MY_OBJECT_NAME)

Grep the INDEX_ID for the the table using `curl` and `jq`.


In [ ]:
%%bash -s "$MY_OBJECT_NAME" "$MY_HOST_IPv4"  "$MY_DB_NAME"  "$MY_TSERVER_WEBSERVER_PORT"  --out MY_INDEX_ID
OBJECT_NAME=$( echo "${1}" | tr -d " ")
HOST_IPv4=$( echo "${2}" | tr -d " ")
DB_NAME=$( echo "${3}" | tr -d " ")
TSERVER_WEBSERVER_PORT=$( echo "${4}" | tr -d " ")


MY_URL="http://${HOST_IPv4}:${TSERVER_WEBSERVER_PORT}/metrics"

MY_INDEX_ID=`curl -s --compressed ${MY_URL} | jq -r 'limit(1;  .[] | select(.attributes.namespace_name=="'${DB_NAME}'" and .type=="tablet" and .attributes.table_name=="'${OBJECT_NAME}'") |  .attributes.table_id) '`

echo ${MY_INDEX_ID}

Store the index_id for the index.

In [ ]:
%store MY_INDEX_ID
print(MY_INDEX_ID)

Get the tablet_id for the tablet leader for the select node host.

In [ ]:
%%bash -s "$MY_OBJECT_NAME" "$MY_HOST_IPv4" --out MY_INDEX_TABLET_ID
OBJECT_NAME=$( echo "${1}" | tr -d " ")
HOST_IPv4=$( echo "${2}" | tr -d " ")

MY_URL="http://${HOST_IPv4}:8200/metrics"

MY_INDEX_TABLET_ID=`curl -s --compressed ${MY_URL} | jq --raw-output ' .[] | select(.attributes.namespace_name=="ks_ybu" and .type=="tablet" and .attributes.table_name=="'$OBJECT_NAME'") | {tablet_id: .id, metrics: .metrics[] | select(.name == ("is_raft_leader") ) | select(.value == 1) } | select(.tablet_id) | {tablet_id} | .tablet_id '`

echo ${MY_INDEX_TABLET_ID}

Store the tablet_id for the tablet leader.

In [ ]:
%store MY_INDEX_TABLET_ID
print(MY_INDEX_TABLET_ID)

Flush the WAL file to a SST file for the given index_id.

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_HOST_IPv4" "$MY_INDEX_ID"  # Import file path of Yugabyte and DB name
YB_PATH=$( echo "${1}" | tr -d " ")
HOST_IPv4=$( echo "${2}" | tr -d " ")
INDEX_ID=$( echo "${3}" | tr -d " ")
cd $YB_PATH/bin

./yb-admin -init_master_addrs ${HOST_IPv4}:7100 flush_table_by_id ${INDEX_ID} 600

Dump and decode the SST file in human-readable form.

> Note:
>
> If the following does **NOT** dump the SST file, it is most likely that there are not any rows written to this tablet. To resolve this issue, you need to select a different Tablet Server host. Return back to **Select a YB-TServer host** and select a different node host.

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_YB_PATH_DATA" "$MY_INDEX_ID" "$MY_INDEX_TABLET_ID" # Import file path of Yugabyte and DB name
YB_PATH=$( echo "${1}" | tr -d " ")
YB_PATH_DATA=$( echo "${2}" | tr -d " ")
INDEX_ID=$( echo "${3}" | tr -d " ")
INDEX_TABLET_ID=$( echo "${4}" | tr -d " ")

cd $YB_PATH/bin/

INDEX_ID_PATH=${YB_PATH_DATA}/node-1/disk-1/yb-data/tserver/data/rocksdb/table-${INDEX_ID}/tablet-${INDEX_TABLET_ID}

# ls -l  ${TABLE_ID_PATH}

./sst_dump --command=scan --file=${INDEX_ID_PATH} --output_format=decoded_regulardb 

The DocKey consists of the partition key hash, the partition key, and the clustering keys. 

The index scan query begins by accessing a single tablet for the index. The seek operation reads data from the SST file for the index tablet using the partition key hash. Because the brand column contains non-unique values, there may be multiple seeks in the related SST file. The seek operation gathers the DocKeys. The DocKeys contains the primary key values for the index table. Using this list, a second operation accesses the index table tablets. When there is more than a single index table tablet, this is a batch operation. Often, the index and table tablets reside on different nodes. This means that the batch operation requires at least one or more remote procedure calls to one or more nodes in the cluster. The goal with the batch operation is to optimize the number of seek operations for a given table tablet by using a list of primary keys that fall within the tablet hash value range. Although this query is not as costly as a sequential scan query, it does require accessing one index tablet and at least one tablet for the index table. 

## Secondary index: `Index Only Scan` query plan

 You can define one or more `include` columns to cover a query with the index alone. The term, covering index, describes a secondary index that a query uses to only access the index and not the table. The previous index required the query to access both the tablets for the index and for the table. A covering index only utilizes the index itself for the query. There are some restrictions for defining an include column in a secondary index. The include column needs to be a column with a basic data type.

### Create the secondary index

To begin, drop the secondary index if it exists.

In [3]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME"   # Secondary index
YB_PATH=${1}
DB_NAME=${2}  
cd $YB_PATH/bin

# DB_NAME=ks_ybu
./ycqlsh -r -k $DB_NAME -e "
drop index if exists idx_products_by_category_brand_inc;"

sleep 1;

Run the following cell to create the index the uses the `include` clause.

In [4]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME"   # Secondary index
YB_PATH=${1}
DB_NAME=${2}  
cd $YB_PATH/bin

# DB_NAME=ks_ybu
./ycqlsh -r -k $DB_NAME -e "
create index if not exists idx_products_by_category_brand_inc 
  on tbl_products_by_category (brand) 
  include (description)
  ;"

Use the `describe` keyword  to verify if the index was created for the given table. 


In [5]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME"   # describe table to view index
YB_PATH=${1}
DB_NAME=${2}  
cd $YB_PATH/bin

# DB_NAME=ks_ybu
./ycqlsh -r -k $DB_NAME -e "
  describe tbl_products_by_category;
  "


CREATE TABLE ks_ybu.tbl_products_by_category (
    category text,
    product_name text,
    product_id int,
    brand text,
    price decimal,
    discount int,
    description text,
    gtin text,
    sku_details jsonb,
    PRIMARY KEY (category, product_name, product_id)
) WITH CLUSTERING ORDER BY (product_name ASC, product_id ASC)
    AND default_time_to_live = 0
    AND transactions = {'enabled': 'true'};
CREATE INDEX idx_products_by_category_brand ON ks_ybu.tbl_products_by_category (brand, category, product_name, product_id)
    WITH CLUSTERING ORDER BY (category ASC, product_name ASC, product_id ASC)
    AND transactions = {'enabled': 'true'};
CREATE INDEX idx_products_by_category_brand_inc ON ks_ybu.tbl_products_by_category (brand, category, product_name, product_id) INCLUDE (description)
    WITH CLUSTERING ORDER BY (category ASC, product_name ASC, product_id ASC)
    AND transactions = {'enabled': 'true'};



There are now two indexes for the table. The index key  for the `idx_products_by_category_brand_inc` index consists of a partition key and zero or more clustering keys. In the previous example, you can see that the index key is `brand`. The index inherits the primary key from the index table. The clustering keys are `category`,`product_name`, and `product_id`.  In addition, the `description` column is added to the index.

### View query plan for a secondary index
Run the following to view the plan.

In [6]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME"   # new query plan
YB_PATH=${1}
DB_NAME=${2}  
cd $YB_PATH/bin

# DB_NAME=ks_ybu
./ycqlsh -r -k $DB_NAME -e "
  explain select  category, brand, product_name, description, product_id 
  from tbl_products_by_category 
  where brand=?;
"  


 QUERY PLAN
----------------------------------------------------------------------------------------------------
 Index Only Scan using ks_ybu.idx_products_by_category_brand_inc on ks_ybu.tbl_products_by_category
   Key Conditions: (brand = :brand)                                                                



#### Index Only Scan
The introduction of a new covering index results in a new query plan. This query plan contains one node. The action is an index only scan. Key Conditions is a sub-action that specifies the use of the partition key for the index. The equality operator indicates that the internal operation is to locate a specific partition key on a single tablet. The index contains the partition key for the index, the primary key columns of the index table, and any included columns from the index table. In this example, the include column is the description column from the index table.

### Index tablet and `index only scan` query
To learn more about why this is an  `index only scan` query, take a look at how the YugabyteDB stores data for this index.

#### Select a YB-TServer host
Set the host variable for one of the nodes. All three nodes in the cluster are running a Tablet Server (YB-TServer). You can comment/uncomment lines 7-9 as needed.


In [7]:
%%bash -s "$MY_HOST_IPv4_01" "$MY_HOST_IPv4_02" "$MY_HOST_IPv4_03" --out MY_HOST_IPv4
HOST_IPv4_01=$( echo "${1}" | tr -d " ")
HOST_IPv4_02=$( echo "${2}" | tr -d " ")
HOST_IPv4_03=$( echo "${3}" | tr -d " ")

# change the hosts for different tablet leaders
MY_HOST_IPv4=$HOST_IPv4_01
#MY_HOST_IPv4=$HOST_IPv4_02
#MY_HOST_IPv4=$HOST_IPv4_03

echo ${MY_HOST_IPv4}

Store the select host variable.

In [8]:
%store MY_HOST_IPv4
print(MY_HOST_IPv4)

Stored 'MY_HOST_IPv4' (str)
127.0.0.1



Save the table name as a variable.

In [9]:
MY_OBJECT_NAME="idx_products_by_category_brand_inc"
%store MY_OBJECT_NAME
print(MY_OBJECT_NAME)

Stored 'MY_OBJECT_NAME' (str)
idx_products_by_category_brand_inc


Grep the INDEX_ID for the the table using `curl` and `jq`.


In [10]:
%%bash -s "$MY_OBJECT_NAME" "$MY_HOST_IPv4"  "$MY_DB_NAME"  "$MY_TSERVER_WEBSERVER_PORT"  --out MY_INDEX_ID
OBJECT_NAME=$( echo "${1}" | tr -d " ")
HOST_IPv4=$( echo "${2}" | tr -d " ")
DB_NAME=$( echo "${3}" | tr -d " ")
TSERVER_WEBSERVER_PORT=$( echo "${4}" | tr -d " ")


MY_URL="http://${HOST_IPv4}:${TSERVER_WEBSERVER_PORT}/metrics"

MY_INDEX_ID=`curl -s --compressed ${MY_URL} | jq -r 'limit(1;  .[] | select(.attributes.namespace_name=="'${DB_NAME}'" and .type=="tablet" and .attributes.table_name=="'${OBJECT_NAME}'") |  .attributes.table_id) '`

echo ${MY_INDEX_ID}

Store the index_id for the index.

In [11]:
%store MY_INDEX_ID
print(MY_INDEX_ID)

Stored 'MY_INDEX_ID' (str)
c905c879a79d48d3928805bfd89922ae



Get the tablet_id for the tablet leader for the select node host.

In [12]:
%%bash -s "$MY_OBJECT_NAME" "$MY_HOST_IPv4" --out MY_INDEX_TABLET_ID
OBJECT_NAME=$( echo "${1}" | tr -d " ")
HOST_IPv4=$( echo "${2}" | tr -d " ")

MY_URL="http://${HOST_IPv4}:8200/metrics"

MY_INDEX_TABLET_ID=`curl -s --compressed ${MY_URL} | jq --raw-output ' .[] | select(.attributes.namespace_name=="ks_ybu" and .type=="tablet" and .attributes.table_name=="'$OBJECT_NAME'") | {tablet_id: .id, metrics: .metrics[] | select(.name == ("is_raft_leader") ) | select(.value == 1) } | select(.tablet_id) | {tablet_id} | .tablet_id '`

echo ${MY_INDEX_TABLET_ID}

Store the tablet_id for the tablet leader.

In [13]:
%store MY_INDEX_TABLET_ID
print(MY_INDEX_TABLET_ID)

Stored 'MY_INDEX_TABLET_ID' (str)
a10b1a1a4f3940c491e36ac9df768064



Flush the WAL file to a SST file for the given index_id.

In [14]:
%%bash -s "$MY_YB_PATH" "$MY_HOST_IPv4" "$MY_INDEX_ID"  # Import file path of Yugabyte and DB name
YB_PATH=$( echo "${1}" | tr -d " ")
HOST_IPv4=$( echo "${2}" | tr -d " ")
INDEX_ID=$( echo "${3}" | tr -d " ")
cd $YB_PATH/bin

./yb-admin -init_master_addrs ${HOST_IPv4}:7100 flush_table_by_id ${INDEX_ID} 600

Flushed [c905c879a79d48d3928805bfd89922ae] tables.


Dump and decode the SST file in human-readable form.

> Note:
>
> If the following does **NOT** dump the SST file, it is most likely that there are not any rows written to this tablet. To resolve this issue, you need to select a different Tablet Server host. Return back to **Select a YB-TServer host** and select a different node host.

In [15]:
%%bash -s "$MY_YB_PATH" "$MY_YB_PATH_DATA" "$MY_INDEX_ID" "$MY_INDEX_TABLET_ID" # Import file path of Yugabyte and DB name
YB_PATH=$( echo "${1}" | tr -d " ")
YB_PATH_DATA=$( echo "${2}" | tr -d " ")
INDEX_ID=$( echo "${3}" | tr -d " ")
INDEX_TABLET_ID=$( echo "${4}" | tr -d " ")

cd $YB_PATH/bin/

INDEX_ID_PATH=${YB_PATH_DATA}/node-1/disk-1/yb-data/tserver/data/rocksdb/table-${INDEX_ID}/tablet-${INDEX_TABLET_ID}

# ls -l  ${TABLE_ID_PATH}

./sst_dump --command=scan --file=${INDEX_ID_PATH} --output_format=decoded_regulardb 

from [] to []
Process /Users/seth/yugabyte-data/node-1/disk-1/yb-data/tserver/data/rocksdb/table-c905c879a79d48d3928805bfd89922ae/tablet-a10b1a1a4f3940c491e36ac9df768064/000010.sst
Sst file format: block-based
SubDocKey(DocKey(0xb464, ["Yeah"], ["H20", "Talc 5", 62362]), [SystemColumnId(0); HT{ physical: 1672699605559607 }]) -> null
SubDocKey(DocKey(0xb464, ["Yeah"], ["H20", "Talc 5", 62362]), [ColumnId(4); HT{ physical: 1672699605559607 w: 1 }]) -> "1 liter"
SubDocKey(DocKey(0xb464, ["Yeah"], ["Hip", "Tute 20", 62361]), [SystemColumnId(0); HT{ physical: 1672699605559607 w: 2 }]) -> null
SubDocKey(DocKey(0xb464, ["Yeah"], ["Hip", "Tute 20", 62361]), [ColumnId(4); HT{ physical: 1672699605559607 w: 3 }]) -> "front waist pack"
SubDocKey(DocKey(0xce08, ["Climb"], ["Day", "Turn 24", 12452]), [SystemColumnId(0); HT{ physical: 1672699605559607 w: 4 }]) -> null
SubDocKey(DocKey(0xce08, ["Climb"], ["Day", "Turn 24", 12452]), [ColumnId(4); HT{ physical: 1672699605559607 w: 5 }]) -> "day pack"
Su

A query plan with an index-only-scan operation accesses a single tablet for the index. The seek operation reads data from the SST file for the index tablet. Depending on the number of products for a given brand, there may be multiple seeks. However, because brand is the partition key for this index, the query locates the tablet with the partition hash key almost immediately. The partition hash key for the index is the DocKey hash. The DocKey consists of the index partition key hash, the index partition key, and clustering keys. The clustering keys are the primary key columns of the index table. The DocKey maps to subdocuments. The subdocuments in this example are the included columns:  description. Each subdocument contains a column value. The covering index efficiently processes the query without needing to access the data for the the index table.

## Unique Indexes
You can disallow a column from having duplicate values by using a unique constraint as shown in the following cell.

In [16]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME"   # describe table to view index
YB_PATH=${1}
DB_NAME=${2}  
cd $YB_PATH/bin

# DB_NAME=ks_ybu
./ycqlsh -r -k $DB_NAME -e "
  create unique index idx_products_by_category_unq
  on tbl_products_by_category (gtin);
"  
  
./bin/ycqlsh --execute "
  desc tbl_products_by_category;
"  
# This statement will cause an error since duplicate values for the 
# product_id are not allowed, 87c7624a-4af5-4347-922d-ab43ab32476b.
./bin/ycqlsh --execute "
  INSERT INTO ks_ybu.tbl_products_by_category (
    product_name, 
    description, 
    price, 
    category,
    product_id 
  ) VALUES (
    'Guard dogs',
    'Doberman Pinchers',
    643.99,
    'Security',
    87c7624a-4af5-4347-922d-ab43ab32476b
  );
"

bash: line 11: ./bin/ycqlsh: No such file or directory
bash: line 16: ./bin/ycqlsh: No such file or directory


CalledProcessError: Command 'b'YB_PATH=${1}\nDB_NAME=${2}  \ncd $YB_PATH/bin\n\n# DB_NAME=ks_ybu\n./ycqlsh -r -k $DB_NAME -e " CREATE UNIQUE INDEX idx_unique_product_by_id \n  ON ks_ybu.tbl_products_by_category(product_id)\n  INCLUDE (description);\n"  \n  \n./bin/ycqlsh --execute "\n  DESC ks_ybu.tbl_products_by_category\n"  \n# This statement will cause an error since duplicate values for the \n# product_id are not allowed, 87c7624a-4af5-4347-922d-ab43ab32476b.\n./bin/ycqlsh --execute "\n  INSERT INTO ks_ybu.tbl_products_by_category (\n    product_name, \n    description, \n    price, \n    category,\n    product_id \n  ) VALUES (\n    \'Guard dogs\',\n    \'Doberman Pinchers\',\n    643.99,\n    \'Security\',\n    87c7624a-4af5-4347-922d-ab43ab32476b\n  );\n"\n'' returned non-zero exit status 127.

>**Important:** Notice the error in the result set: `Execution Error. Duplicate value disallowed by unique index idx_unique_product_by_id`. This verifies that the unique index that was created for the `product_id` is preventing a duplicate value for this attribute. Currently the same `product_id` identitifies a backpack and cannot be used again for a different product.

## Partial Indexes

Use a range partition to reduce the amount of data that requires scanning.
In this index, you will partition all products over $30 in the Office Supply category. 

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH

# Query Plan
./bin/ycqlsh --execute "
  CREATE INDEX ON ks_ybu.tbl_products_by_category(price) 
  INCLUDE (description, product_name) 
  WHERE price > 30 and category = 'Office Supplies';
"

./bin/ycqlsh --execute "
  DESC ks_ybu.tbl_products_by_category
"  

### Specific Indexes
An index can be created that specifies a particular value of an attribute. For instance, suppose that a popular item in our ecommerce site is searched for constantly. It may be beneficial to create an index that satisfies this query quickly and efficiently. In the following cell, there is an example of an index for a popular product, batteries.

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH

# Specific Index
./bin/ycqlsh --execute "
  CREATE INDEX idx_batteries_desc 
  ON ks_ybu.tbl_products_by_category (description) 
  INCLUDE (product_name) 
  WHERE description='Batteries';
"
#  Index Only Scan
./bin/ycqlsh --execute "
  EXPLAIN SELECT 
    description, 
    product_name, 
    product_id, 
    category, 
    price 
  FROM ks_ybu.tbl_products_by_category 
  WHERE description='Batteries';
"  

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH

#  Seq Scan
./bin/ycqlsh --execute "
  EXPLAIN SELECT description, product_name, product_id, category, price FROM ks_ybu.tbl_products_by_category 
  WHERE description='Hotdogs';
"  

>**Important:** Note that in the previous cell, an index only scan will occur only if the WHERE clause predicate returns the same expression as originally stated in the index creation statement.

Run the following statement to verify that a query to a different expression will not use the index to satisfy the query.

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH

#  Seq Scan
./bin/ycqlsh --execute "
  EXPLAIN SELECT description, product_name, product_id, category, price FROM ks_ybu.tbl_products_by_category 
  WHERE description=?;
"  

In the preceding cell, note the syntax necessary to insert a JSON object into the table. Outside the curly brackets are single quotes, where inside the brackets there are double quotes.

If the row is still visible, wait a few more seconds to run the preceding cell. This will verify that the row has expired as expected.

---
# All done!
In this lab, you completed the following:

- Setup
  - Created the `ks_ybu` database with `ycqlsh`
  - Created tables and loaded data using DDL and DML scripts